In [1]:
import requests

In [2]:
r = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?id=013')

In [3]:
print(r)

<Response [200]>


In [4]:
r.text

'<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="13">\n         <thumbnail>https://cf.geekdo-images.com/thumb/img/g8LvJsd2oLAub6AEKN_Xpdoi8Sw=/fit-in/200x150/pic2419375.jpg</thumbnail>\n      <image>https://cf.geekdo-images.com/original/img/A-0yDJkve0avEicYQ4HoNO-HkK8=/0x0/pic2419375.jpg</image>\n                                     \t\t\t\t\n\t\t\t\t<name type="primary" sortindex="1" value="Catan" />\n\t\t\t\n\t\t\t\t\t\t                               \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="CATAN" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Catan (Колонизаторы)" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Catan: Das Spiel" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Catan: El Juego" />\n\t\t\t    \t\t\t\t\n\t\t\t\t<name type="alternate" sortindex="1" value="Catan: Gra planszowa" />\n\t\t\t

In [5]:
import requests
from xml.etree import ElementTree as ET

response = requests.get('https://www.boardgamegeek.com/xmlapi2/thing?id=013')

tree = ET.fromstring(response.content)

In [6]:
print(tree)

<Element 'items' at 0x000001EC518D0A48>


In [8]:
tree.tag

'items'

In [9]:
tree.attrib

{'termsofuse': 'https://boardgamegeek.com/xmlapi/termsofuse'}

In [10]:
for child in tree:
    print(child.tag, child.attrib)

item {'type': 'boardgame', 'id': '13'}


In [12]:
# List of all the elements
[elem.tag for elem in tree.iter()]

['items',
 'item',
 'thumbnail',
 'image',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'name',
 'description',
 'yearpublished',
 'minplayers',
 'maxplayers',
 'poll',
 'results',
 'result',
 'result',
 'result',
 'results',
 'result',
 'result',
 'result',
 'results',
 'result',
 'result',
 'result',
 'results',
 'result',
 'result',
 'result',
 'results',
 'result',
 'result',
 'result',
 'playingtime',
 'minplaytime',
 'maxplaytime',
 'minage',
 'poll',
 'results',
 'result',
 'result',
 'result',
 'result',
 'result',
 'result',


In [13]:
# See the whole text in XML format
print(ET.tostring(tree, encoding='utf8').decode('utf8'))

<?xml version='1.0' encoding='utf8'?>
<items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item id="13" type="boardgame">
         <thumbnail>https://cf.geekdo-images.com/thumb/img/g8LvJsd2oLAub6AEKN_Xpdoi8Sw=/fit-in/200x150/pic2419375.jpg</thumbnail>
      <image>https://cf.geekdo-images.com/original/img/A-0yDJkve0avEicYQ4HoNO-HkK8=/0x0/pic2419375.jpg</image>
                                     				
				<name sortindex="1" type="primary" value="Catan" />
			
						                               				
				<name sortindex="1" type="alternate" value="CATAN" />
			    				
				<name sortindex="1" type="alternate" value="Catan (Колонизаторы)" />
			    				
				<name sortindex="1" type="alternate" value="Catan: Das Spiel" />
			    				
				<name sortindex="1" type="alternate" value="Catan: El Juego" />
			    				
				<name sortindex="1" type="alternate" value="Catan: Gra planszowa" />
			    				
				<name sortindex="1" type="alternate" value="Catan: Il Gioco" />
			    				


In [30]:
# Variable to find the primary name of the boardgame
primary = "'type': 'primary'"

# Loop to find the name of the boardgame in the tree
for boardgame in tree.iter('name'):
    if primary in str(boardgame.attrib):
        boardgame_name = boardgame.attrib
        print(boardgame_name)

{'type': 'primary', 'sortindex': '1', 'value': 'Catan'}


In [31]:
boardgame_name

{'sortindex': '1', 'type': 'primary', 'value': 'Catan'}